Сначала всё подключим: mystem, pymorphy, nltk, json. И возьмём текст (и его кусочек для экспериментов)

In [31]:
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import os

os.environ["MYSTEM_BIN"] = "C:\\mystem.exe"
ms = Mystem()

morph = MorphAnalyzer()

tokenizer = RegexpTokenizer(r'\w+')

with open('book.txt', 'r', encoding='utf-8') as f:
    text = f.read()
with open('book.txt', 'r', encoding='utf-8') as f:
    text_lines = f.readlines()

with open('book_part.txt', 'r', encoding='utf-8') as f:
    text_short = f.read()
with open('book_part.txt', 'r', encoding='utf-8') as f:
    text_short_lines = f.readlines()


Вот в этой клеточке замерим время работы майстема

In [2]:
%%time

lemmas = ms.lemmatize(text_short)


Wall time: 4min 23s


А вот в этой нлтк+пайморфи

In [3]:
%%time

tokens = word_tokenize(text_short)
analyze = []
for token in tokens:
    analyze.append(morph.parse(token)[0])


Wall time: 848 ms


На windows модуль mystem работает ооочень медленно, в отличие от pymorphy.

Сделаем json с леммами

In [64]:
lemmas = []
for line in text_lines:
    # не берём последний элемент списка, потому что это '\n'
    lemmas.extend(ms.lemmatize(line)[:-1])

with open('lemmas_mystem.json', 'w', encoding='utf-8') as file:
    json.dump(lemmas, file, ensure_ascii=False)


Теперь посмотрим на пайморфи и нлтк.

In [4]:
tokens = word_tokenize(text)
# нужный вид - {"lemma": "конь", "word": "коня", "pos": "NOUN"}
with open('pymorphy_analyze.jsonl', 'w', encoding='utf-8') as f:
    for token in tokens:
        analyze = morph.parse(token)[0]
        analyze_dict = {}
        analyze_dict['lemma'] = analyze.normal_form
        analyze_dict['word'] = token
        analyze_dict['pos'] = analyze.tag.POS
        f.write(json.dumps(analyze_dict, ensure_ascii=False)+'\n')


Вычислим частотность частей речи

In [12]:
freq = {}
# счётчик слов без части речи, чтобы отфильтровать знаки препинания, в основном
waste = 0
for token in tokens:
    pos_tag = morph.parse(token)[0].tag.POS
    if pos_tag in freq:
        freq[pos_tag] += 1
    elif pos_tag is not None:
        freq[pos_tag] = 1
    else:
        waste += 1
for pos in freq:
    freq[pos] = freq[pos]/(len(tokens) - waste)
for key in sorted(freq, key=freq.get, reverse=True):
    print(key, freq[key])


NOUN 0.23692669331932437
VERB 0.17151786588786294
CONJ 0.10103375508641338
PREP 0.09857624945462964
ADJF 0.09068730021636734
NPRO 0.08327026329145483
ADVB 0.07747375544257361
PRCL 0.06899714181410216
ADJS 0.021939470567808456
INFN 0.020835373834688226
PRED 0.006348556215441327
PRTF 0.00595677995530189
NUMR 0.004674603103936461
COMP 0.004478714973866742
GRND 0.0035437943530794505
INTJ 0.0023239455430998407
PRTS 0.0014157369400493282


Чаще всего встречаются существительные и глаголы, также частотны разные служебные части речи
Теперь найдём самые частотные глаголы и наречия

In [17]:
import collections
verbs = []
adverbs = []
for token in tokens:
    analyze = morph.parse(token)[0]
    pos_tag = analyze.tag.POS
    if pos_tag == 'VERB':
        verbs.append(analyze.normal_form)
    if pos_tag == 'ADVB':
        adverbs.append(analyze.normal_form)

counter_verb = collections.Counter(verbs)
counter_advb = collections.Counter(adverbs)
print(counter_verb)
print(counter_advb)


Counter({'быть': 1391, 'сказать': 1154, 'мочь': 457, 'знать': 410, 'спросить': 303, 'стать': 243, 'говорить': 228, 'хотеть': 193, 'подумать': 166, 'смотреть': 157, 'пойти': 152, 'идти': 150, 'стоять': 143, 'бояться': 121, 'думать': 118, 'посмотреть': 104, 'казаться': 103, 'взять': 98, 'увидеть': 93, 'оказаться': 90, 'сидеть': 89, 'уйти': 89, 'понять': 88, 'вздохнуть': 84, 'отозваться': 84, 'давать': 80, 'жить': 79, 'объяснить': 77, 'сделать': 76, 'улыбнуться': 76, 'проговорить': 76, 'видеть': 71, 'понимать': 69, 'прийти': 69, 'остаться': 66, 'сесть': 64, 'играть': 63, 'помнить': 62, 'заметить': 57, 'дать': 55, 'лежать': 54, 'успеть': 54, 'вспомнить': 52, 'ответить': 52, 'оглянуться': 52, 'молчать': 52, 'поднять': 51, 'усмехнуться': 51, 'бывать': 51, 'послать': 50, 'вернуться': 50, 'появиться': 49, 'ждать': 48, 'решить': 48, 'начать': 47, 'верить': 47, 'делать': 46, 'получиться': 46, 'случиться': 45, 'найти': 45, 'прошептать': 45, 'встать': 45, 'кивнуть': 43, 'удивиться': 43, 'любить': 

Никаких "особенных" глаголов в первой двадцатке не заметно, но можно заметить небольшое влияние приключенческого жанра ("сказать", "бояться", "взять"). Аналогично и в наречиях, много разных наречий для описания действия, которых в таком жанре много ("тут", "тогда", "здесь", "сейчас").

In [11]:
ngram_list = []
for line in text_lines[:1000]:
    line_tokens = tokenizer.tokenize(line)
    n_grams = list(ngrams(line_tokens, 2))
    ngram_list.extend(n_grams)
print(collections.Counter(ngram_list))


Counter({('сказала', 'Данка'): 16, ('сказал', 'Яр'): 13, ('не', 'было'): 10, ('сказал', 'Алька'): 10, ('Ну', 'и'): 9, ('и', 'что'): 8, ('посмотрел', 'на'): 7, ('спросил', 'Яр'): 7, ('к', 'Яру'): 6, ('кто', 'то'): 5, ('я', 'не'): 5, ('Ну', 'что'): 5, ('его', 'за'): 5, ('Это', 'был'): 5, ('все', 'же'): 5, ('Данка', 'и'): 5, ('сказал', 'Чита'): 5, ('он', 'не'): 5, ('что', 'он'): 5, ('Алька', 'и'): 5, ('и', 'Чита'): 4, ('что', 'не'): 4, ('подумал', 'Яр'): 4, ('Это', 'не'): 4, ('то', 'не'): 4, ('что', 'ли'): 4, ('какой', 'то'): 4, ('что', 'то'): 4, ('А', 'что'): 4, ('там', 'в'): 4, ('в', 'воду'): 4, ('хотя', 'бы'): 4, ('Но', 'тут'): 4, ('тут', 'же'): 4, ('Яр', 'Я'): 4, ('и', 'не'): 4, ('Яра', 'за'): 4, ('шагнул', 'к'): 4, ('сказал', 'он'): 4, ('У', 'нас'): 4, ('у', 'него'): 4, ('как', 'это'): 4, ('сказал', 'Игнатик'): 4, ('Игнатик', 'и'): 4, ('похожий', 'на'): 3, ('Яр', 'вспомнил'): 3, ('вспомнил', 'как'): 3, ('Кто', 'то'): 3, ('Раз', 'два'): 3, ('не', 'очень'): 3, ('и', 'так'): 3, ('Когда'

Мой ноутбук здесь отказался выдавать все нграммы без последующего падения, поэтому здесь представлен только кусочек, но в целом тенденция сохраняется: много всякого "X сказал", потому что в книге много диалогов. Ещё много комбинаций с отрицанием и союзами/предлогами.

In [13]:
ngram_list = []
for line in text_lines[:2000]:
    line_tokens = tokenizer.tokenize(line)
    n_grams = list(ngrams(line_tokens, 3))
    ngram_list.extend(n_grams)
print(collections.Counter(ngram_list))


Counter({('в', 'самом', 'деле'): 7, ('Ну', 'и', 'что'): 5, ('А', 'что', 'за'): 5, ('Игнатик', 'и', 'Алька'): 5, ('Раз', 'два', 'три'): 3, ('Вот', 'это', 'да'): 3, ('спросил', 'Яр', 'у'): 3, ('то', 'не', 'так'): 3, ('посмотрел', 'на', 'Яра'): 3, ('Яр', 'понял', 'что'): 3, ('Яра', 'за', 'руку'): 3, ('Яру', 'в', 'лицо'): 3, ('посмотрел', 'на', 'него'): 3, ('В', 'самом', 'деле'): 3, ('Тик', 'посмотрел', 'на'): 2, ('то', 'же', 'самое'): 2, ('отрываясь', 'от', 'книги'): 2, ('на', 'Данку', 'и'): 2, ('Яр', 'вспомнил', 'как'): 2, ('вспомнил', 'как', 'он'): 2, ('два', 'три', 'четыре'): 2, ('три', 'четыре', 'пять'): 2, ('Не', 'надо', 'про'): 2, ('на', 'спинку', 'скамьи'): 2, ('Чита', 'поднял', 'голову'): 2, ('Это', 'был', 'старый'): 2, ('Яр', 'у', 'Данки'): 2, ('не', 'было', 'И'): 2, ('сказал', 'Яр', 'Я'): 2, ('Яр', 'взглянул', 'на'): 2, ('взглянул', 'на', 'часы'): 2, ('Данка', 'и', 'Тик'): 2, ('был', 'какой', 'то'): 2, ('Но', 'тут', 'же'): 2, ('Яр', 'подумал', 'что'): 2, ('висит', 'в', 'субпрост

Здесь скорее устойчивые выражения преобладают "в самом деле", "ну и что" и т.д. Но остальные в основном - сочетания с отрицанием и союзами/предлогами, как и в прошлый раз. (и смешное "раз два три")